In [1]:
import torch
import torch.nn.functional as F
from torch.optim.lr_scheduler import StepLR, MultiStepLR
import numpy as np
import torch.nn as nn
from math import *

In [2]:
torch.cuda.set_device(1)

In [3]:
torch.set_default_tensor_type('torch.DoubleTensor') # 设置浮点类型为 torch.float64

In [4]:
# 定义激活函数: swish(x)
def acti(x):
    return x*torch.sigmoid(x)  

In [5]:
# 定义网络结构
class DeepRitzNet(torch.nn.Module):
    def __init__(self, input_width, layer_width):
        super(DeepRitzNet, self).__init__()
        self.linear_in = torch.nn.Linear(input_width, layer_width)
        self.linear1 = torch.nn.Linear(layer_width, layer_width)
        self.linear2 = torch.nn.Linear(layer_width, layer_width)
        self.linear3 = torch.nn.Linear(layer_width, layer_width)
        self.linear4 = torch.nn.Linear(layer_width, layer_width)
        self.linear5 = torch.nn.Linear(layer_width, layer_width)
        self.linear6 = torch.nn.Linear(layer_width, layer_width)
        self.linear_out = torch.nn.Linear(layer_width, 1)

    def forward(self, x):
        y = self.linear_in(x) # fully connect layer
        y = y + acti(self.linear2(acti(self.linear1(y)))) # residual block 1
        y = y + acti(self.linear4(acti(self.linear3(y)))) # residual block 2
        y = y + acti(self.linear6(acti(self.linear5(y)))) # residual block 3
        output = self.linear_out(y) # fully connect layer
        return output

In [6]:
dimension = 16

In [7]:
# exact solution
def u_ex(x):  
    x_temp = torch.cos(pi*x)
    u_x = (x_temp.sum(1)).reshape([x.size()[0], 1]) # x_temp.sum(1) 按行求和
    return u_x

In [8]:
def f(x):
    x_temp = torch.cos(pi*x)
    f_x = 2*pi**2*(x_temp.sum(1)).reshape([x.size()[0], 1]) # x_temp.sum(1) 按行求和
    return f_x

In [9]:
# Deep Ritz Method
def DRM(x):
    u_hat = model(x)
    ux = torch.zeros(x.size()[0], dimension).cuda()
    step_size = 0.0001
    for i in range(dimension):
        dx = torch.zeros(x.size()[0], dimension).cuda()
        dx[:, i] = torch.ones(x.size()[0])
        ux[:, i] = (model(x+step_size*dx) - model(x-step_size*dx))[:,0]/step_size/2
    uxsq = (torch.sum(ux**2, dim = 1)).reshape([x.size()[0], 1]) # dim = 1 按行求和
    f_temp = f(x)
    part_1 = torch.sum(0.5 * uxsq + 0.5*pi**2*u_hat**2 - f_temp*u_hat)/x.size()[0]
    
    Nb = 100
    xa1 = 2*torch.rand(Nb, dimension) - 1.0
    xa1[:, 0] = -torch.ones(Nb)
    
    xa2 = 2*torch.rand(Nb, dimension) - 1.0
    xa2[:, 0:16] = xa1[:, 0:16]
    xa2[:, 0] = torch.ones(Nb)
    
    xb1 = 2*torch.rand(Nb, dimension) - 1.0
    xb1[:, 1] = -torch.ones(Nb)
    
    xb2 = 2*torch.rand(Nb, dimension) - 1.0
    xb2[:, 0:16] = xb1[:, 0:16] 
    xb2[:, 1] = torch.ones(Nb)
    
    xc1 = 2*torch.rand(Nb, dimension) - 1.0
    xc1[:, 2] = -torch.ones(Nb)
    
    xc2 = 2*torch.rand(Nb, dimension) - 1.0
    xc2[:, 0:16] = xc1[:, 0:16]
    xc2[:, 2] = torch.ones(Nb)
    
    xd1 = 2*torch.rand(Nb, dimension) - 1.0
    xd1[:, 3] = -torch.ones(Nb)
    
    xd2 = 2*torch.rand(Nb, dimension) - 1.0
    xd2[:, 0:16] = xd1[:, 0:16]
    xd2[:, 3] = torch.ones(Nb)
    
    xe1 = (2*torch.rand(Nb, dimension) - 1.0)
    xe1[:, 4] = -torch.ones(Nb)
    
    xe2 = (2*torch.rand(Nb, dimension) - 1.0)
    xe2[:, 0:16] = xe1[:, 0:16]
    xe2[:, 4] = torch.ones(Nb)
    
    xf1 = 2*torch.rand(Nb, dimension) - 1.0
    xf1[:, 5] = -torch.ones(Nb)
    
    xf2 = 2*torch.rand(Nb, dimension) - 1.0
    xf2[:, 0:16] = xf1[:, 0:16]
    xf2[:, 5] = torch.ones(Nb)
    
    xg1 = 2*torch.rand(Nb, dimension) - 1.0
    xg1[:, 6] = -torch.ones(Nb)
    
    xg2 = 2*torch.rand(Nb, dimension) - 1.0
    xg2[:, 0:16] = xg1[:, 0:16] 
    xg2[:, 6] = torch.ones(Nb)
    
    xh1 = 2*torch.rand(Nb, dimension) - 1.0
    xh1[:, 7] = -torch.ones(Nb)
    
    xh2 = 2*torch.rand(Nb, dimension) - 1.0
    xh2[:, 0:16] = xh1[:, 0:16]
    xh2[:, 7] = torch.ones(Nb)
    
    xi1 = 2*torch.rand(Nb, dimension).cuda() - 1.0  
    xi1[:, 8] = -torch.ones(Nb)
    
    xi2 = 2*torch.rand(Nb, dimension).cuda() - 1.0
    xi2[:, 0:16] = xi1[:, 0:16]
    xi2[:, 8] = torch.ones(Nb)
    
    xj1 = 2*torch.rand(Nb, dimension).cuda() - 1.0 
    xj1[:, 9] = -torch.ones(Nb)
    
    xj2 = 2*torch.rand(Nb, dimension).cuda() - 1.0
    xj2[:, 0:16] = xj1[:, 0:16]
    xj2[:, 9] = torch.ones(Nb)
    
    xk1 = 2*torch.rand(Nb, dimension).cuda() - 1.0
    xk1[:, 10] = -torch.ones(Nb)
    
    xk2 = 2*torch.rand(Nb, dimension).cuda() - 1.0
    xk2[:, 0:16] = xk1[:, 0:16]
    xk2[:, 10] = torch.ones(Nb)
    
    xl1 = 2*torch.rand(Nb, dimension).cuda() - 1.0
    xl1[:, 11] = -torch.ones(Nb)
    
    xl2 = 2*torch.rand(Nb, dimension).cuda() - 1.0
    xl2[:, 0:16] = xl1[:, 0:16]
    xl2[:, 11] = torch.ones(Nb)
    
    xm1 = 2*torch.rand(Nb, dimension).cuda() - 1.0 
    xm1[:, 12] = -torch.ones(Nb)
    
    xm2 = 2*torch.rand(Nb, dimension).cuda() - 1.0
    xm2[:, 0:16] = xm1[:, 0:16]
    xm2[:, 12] = torch.ones(Nb)
    
    xn1 = 2*torch.rand(Nb, dimension).cuda() - 1.0 
    xn1[:, 13] = -torch.ones(Nb)
    
    xn2 = 2*torch.rand(Nb, dimension).cuda() - 1.0
    xn2[:, 0:16] = xn1[:, 0:16]
    xn2[:, 13] = torch.ones(Nb)
    
    xo1 = 2*torch.rand(Nb, dimension).cuda() - 1.0
    xo1[:, 14] = -torch.ones(Nb)
    
    xo2 = 2*torch.rand(Nb, dimension).cuda() - 1.0
    xo2[:, 0:16] = xo1[:, 0:16]
    xo2[:, 14] = torch.ones(Nb)
    
    xp1 = 2*torch.rand(Nb, dimension).cuda() - 1.0
    xp1[:, 15] = -torch.ones(Nb)
    
    xp2 = 2*torch.rand(Nb, dimension).cuda() - 1.0
    xp2[:, 0:16] = xp1[:, 0:16]
    xp2[:, 15] = torch.ones(Nb)
    
    xa1 = xa1.cuda()
    xa2 = xa2.cuda()
    xb1 = xb1.cuda()
    xb2 = xb2.cuda()
    xc1 = xc1.cuda()
    xc2 = xc2.cuda()
    xd1 = xd1.cuda()
    xd2 = xd2.cuda()
    xe1 = xe1.cuda()
    xe2 = xe2.cuda()
    xf1 = xf1.cuda()
    xf2 = xf2.cuda()
    xg1 = xg1.cuda()
    xg2 = xg2.cuda()
    xh1 = xh1.cuda()
    xh2 = xh2.cuda()
    xi1 = xi1.cuda()
    xi2 = xi2.cuda()
    xj1 = xj1.cuda()
    xj2 = xj2.cuda()
    xk1 = xk1.cuda()
    xk2 = xk2.cuda()
    xl1 = xl1.cuda()
    xl2 = xl2.cuda()
    xm1 = xm1.cuda()
    xm2 = xm2.cuda()
    xn1 = xn1.cuda()
    xn2 = xn2.cuda()
    xo1 = xo1.cuda()
    xo2 = xo2.cuda()
    xp1 = xp1.cuda()
    xp2 = xp2.cuda()
    
    pxa = ((model(xa1) - model(xa2))**2).cuda() 
    pxb = ((model(xb1) - model(xb2))**2).cuda()  
    pxc = ((model(xc1) - model(xc2))**2).cuda()  
    pxd = ((model(xd1) - model(xd2))**2).cuda()  
    pxe = ((model(xe1) - model(xe2))**2).cuda() 
    pxf = ((model(xf1) - model(xf2))**2).cuda()  
    pxg = ((model(xg1) - model(xg2))**2).cuda()  
    pxh = ((model(xh1) - model(xh2))**2).cuda()
    pxi = ((model(xi1) - model(xi2))**2).cuda() 
    pxj = ((model(xj1) - model(xj2))**2).cuda()  
    pxk = ((model(xk1) - model(xk2))**2).cuda()  
    pxl = ((model(xl1) - model(xl2))**2).cuda()  
    pxm = ((model(xm1) - model(xm2))**2).cuda() 
    pxn = ((model(xn1) - model(xn2))**2).cuda()  
    pxo = ((model(xo1) - model(xo2))**2).cuda()  
    pxp = ((model(xp1) - model(xp2))**2).cuda()
    
    part_2 = torch.sum((pxa + pxb + pxc + pxd + pxe + pxf + pxg + pxh + pxi + pxj + pxk + pxl + pxm + pxn + pxo + pxp)[:,0])/2/xa1.size()[0]/16
    
    dx1 = torch.zeros(xb1.size()[0], dimension).cuda()  
    dx1[:, 0] = torch.ones(xb1.size()[0])
    dx2 = torch.zeros(xb1.size()[0], dimension).cuda()  
    dx2[:, 1] = torch.ones(xb1.size()[0])
    dx3 = torch.zeros(xb1.size()[0], dimension).cuda()  
    dx3[:, 2] = torch.ones(xb1.size()[0])
    dx4 = torch.zeros(xb1.size()[0], dimension).cuda()  
    dx4[:, 3] = torch.ones(xb1.size()[0])
    dx5 = torch.zeros(xb1.size()[0], dimension).cuda() 
    dx5[:, 4] = torch.ones(xb1.size()[0])
    dx6 = torch.zeros(xb1.size()[0], dimension).cuda() 
    dx6[:, 5] = torch.ones(xb1.size()[0])
    dx7 = torch.zeros(xb1.size()[0], dimension).cuda() 
    dx7[:, 6] = torch.ones(xb1.size()[0])
    dx8 = torch.zeros(xb1.size()[0], dimension).cuda() 
    dx8[:, 7] = torch.ones(xb1.size()[0])
    dx9 = torch.zeros(xb1.size()[0], dimension).cuda() 
    dx9[:, 8] = torch.ones(xb1.size()[0])
    dx10 = torch.zeros(xb1.size()[0], dimension).cuda() 
    dx10[:, 9] = torch.ones(xb1.size()[0])
    dx11 = torch.zeros(xb1.size()[0], dimension).cuda() 
    dx11[:, 10] = torch.ones(xb1.size()[0])
    dx12 = torch.zeros(xb1.size()[0], dimension).cuda() 
    dx12[:, 11] = torch.ones(xb1.size()[0])
    dx13 = torch.zeros(xb1.size()[0], dimension).cuda() 
    dx13[:, 12] = torch.ones(xb1.size()[0])
    dx14 = torch.zeros(xb1.size()[0], dimension).cuda() 
    dx14[:, 13] = torch.ones(xb1.size()[0])
    dx15 = torch.zeros(xb1.size()[0], dimension).cuda() 
    dx15[:, 14] = torch.ones(xb1.size()[0])
    dx16 = torch.zeros(xb1.size()[0], dimension).cuda() 
    dx16[:, 15] = torch.ones(xb1.size()[0])
    
    dxa = (((model(xa1+step_size*dx1) - model(xa1-step_size*dx1))/step_size/2 - (model(xa2+step_size*dx1) - model(xa2-step_size*dx1))/step_size/2)**2).cuda()  
    dxb = (((model(xb1+step_size*dx2) - model(xb1-step_size*dx2))/step_size/2 - (model(xb2+step_size*dx2) - model(xb2-step_size*dx2))/step_size/2)**2).cuda()  
    dxc = (((model(xc1+step_size*dx3) - model(xc1-step_size*dx3))/step_size/2 - (model(xc2+step_size*dx3) - model(xc2-step_size*dx3))/step_size/2)**2).cuda()  
    dxd = (((model(xd1+step_size*dx4) - model(xd1-step_size*dx4))/step_size/2 - (model(xd2+step_size*dx4) - model(xd2-step_size*dx4))/step_size/2)**2).cuda()  
    dxe = (((model(xe1+step_size*dx5) - model(xe1-step_size*dx5))/step_size/2 - (model(xe2+step_size*dx5) - model(xe2-step_size*dx5))/step_size/2)**2).cuda()  
    dxf = (((model(xf1+step_size*dx6) - model(xf1-step_size*dx6))/step_size/2 - (model(xf2+step_size*dx6) - model(xf2-step_size*dx6))/step_size/2)**2).cuda()  
    dxg = (((model(xg1+step_size*dx7) - model(xg1-step_size*dx7))/step_size/2 - (model(xg2+step_size*dx7) - model(xg2-step_size*dx7))/step_size/2)**2).cuda()  
    dxh = (((model(xh1+step_size*dx8) - model(xh1-step_size*dx8))/step_size/2 - (model(xh2+step_size*dx8) - model(xh2-step_size*dx8))/step_size/2)**2).cuda()  
    dxi = (((model(xi1+step_size*dx9) - model(xi1-step_size*dx9))/step_size/2 - (model(xi2+step_size*dx9) - model(xi2-step_size*dx9))/step_size/2)**2).cuda()  
    dxj = (((model(xj1+step_size*dx10) - model(xj1-step_size*dx10))/step_size/2 - (model(xj2+step_size*dx10) - model(xj2-step_size*dx10))/step_size/2)**2).cuda()  
    dxk = (((model(xk1+step_size*dx11) - model(xk1-step_size*dx11))/step_size/2 - (model(xk2+step_size*dx11) - model(xk2-step_size*dx11))/step_size/2)**2).cuda()  
    dxl = (((model(xl1+step_size*dx12) - model(xl1-step_size*dx12))/step_size/2 - (model(xl2+step_size*dx12) - model(xl2-step_size*dx12))/step_size/2)**2).cuda()  
    dxm = (((model(xm1+step_size*dx13) - model(xm1-step_size*dx13))/step_size/2 - (model(xm2+step_size*dx13) - model(xm2-step_size*dx13))/step_size/2)**2).cuda()  
    dxn = (((model(xn1+step_size*dx14) - model(xn1-step_size*dx14))/step_size/2 - (model(xn2+step_size*dx14) - model(xn2-step_size*dx14))/step_size/2)**2).cuda()  
    dxo = (((model(xo1+step_size*dx15) - model(xo1-step_size*dx15))/step_size/2 - (model(xo2+step_size*dx15) - model(xo2-step_size*dx15))/step_size/2)**2).cuda()  
    dxp = (((model(xp1+step_size*dx16) - model(xp1-step_size*dx16))/step_size/2 - (model(xp2+step_size*dx16) - model(xp2-step_size*dx16))/step_size/2)**2).cuda()  
    
    part_3 = torch.sum((dxa + dxb + dxc + dxd + dxe + dxf + dxg + dxh + dxi + dxj + dxk + dxl + dxm + dxn + dxo + dxp)[:,0])/2/xb1.size()[0]/16
    
    lambda1 = 10
    lambda2 = 5
    return part_1 + lambda1 * part_2 + lambda2 * part_3

In [10]:
Data_size = 2000
def Gendata():
    x = 2*torch.rand(Data_size, dimension) - 1.0
    return x.cuda()

In [11]:
# 正态分布初始化参数
def initparam(model, sigma):
    for m in model.modules():
        if isinstance(m, nn.Linear):
            m.weight.data.normal_(0, sigma)
    return model

In [12]:
model = DeepRitzNet(dimension, 32)
model = initparam(model, 0.5)

In [13]:
device = torch.device("cuda:1")
model.to(device)

DeepRitzNet(
  (linear_in): Linear(in_features=16, out_features=32, bias=True)
  (linear1): Linear(in_features=32, out_features=32, bias=True)
  (linear2): Linear(in_features=32, out_features=32, bias=True)
  (linear3): Linear(in_features=32, out_features=32, bias=True)
  (linear4): Linear(in_features=32, out_features=32, bias=True)
  (linear5): Linear(in_features=32, out_features=32, bias=True)
  (linear6): Linear(in_features=32, out_features=32, bias=True)
  (linear_out): Linear(in_features=32, out_features=1, bias=True)
)

In [14]:
import torch.optim as optim
import torch.nn as nn
import time

In [15]:
def relative_error():
    x = Gendata()
    predict = model(x)
    exact = u_ex(x)
    value = torch.sqrt(torch.sum((predict - exact)**2))/torch.sqrt(torch.sum((exact)**2))
    return value

In [16]:
traintime = 100000
error_save = np.zeros(traintime)
optimizer = optim.Adam(model.parameters())

In [17]:
time_start = time.time()
for i in range(traintime):
    optimizer.zero_grad()
    x = Gendata()
    x.requires_grad = True
    losses = DRM(x)
    losses.backward()
    optimizer.step()
    error = relative_error()
    error_save[i] = float(error)
    
    if i % 50 == 0:
        print("current epoch is: ", i)
        print("current loss is: ", losses.detach())
        print("current relative error is: ", error.detach())
        np.save("DRM_relative_error_16D_Periodic.npy", error_save)
np.save("DRM_relative_error_16D_Periodic.npy", error_save)
time_end = time.time()
print('total time is: ', time_end-time_start, 'seconds')

current epoch is:  0
current loss is:  tensor(1116642.2157, device='cuda:1')
current relative error is:  tensor(111.8942, device='cuda:1')
current epoch is:  50
current loss is:  tensor(63876.6092, device='cuda:1')
current relative error is:  tensor(15.5543, device='cuda:1')
current epoch is:  100
current loss is:  tensor(24660.6932, device='cuda:1')
current relative error is:  tensor(8.9912, device='cuda:1')
current epoch is:  150
current loss is:  tensor(12645.8717, device='cuda:1')
current relative error is:  tensor(6.4310, device='cuda:1')
current epoch is:  200
current loss is:  tensor(7860.5624, device='cuda:1')
current relative error is:  tensor(5.2316, device='cuda:1')
current epoch is:  250
current loss is:  tensor(5393.7718, device='cuda:1')
current relative error is:  tensor(4.2530, device='cuda:1')
current epoch is:  300
current loss is:  tensor(3881.8144, device='cuda:1')
current relative error is:  tensor(3.6899, device='cuda:1')
current epoch is:  350
current loss is:  t

current epoch is:  3100
current loss is:  tensor(-3.4168, device='cuda:1')
current relative error is:  tensor(0.9424, device='cuda:1')
current epoch is:  3150
current loss is:  tensor(-2.8712, device='cuda:1')
current relative error is:  tensor(0.9307, device='cuda:1')
current epoch is:  3200
current loss is:  tensor(-5.1203, device='cuda:1')
current relative error is:  tensor(0.9358, device='cuda:1')
current epoch is:  3250
current loss is:  tensor(-2.6776, device='cuda:1')
current relative error is:  tensor(0.9324, device='cuda:1')
current epoch is:  3300
current loss is:  tensor(-5.7563, device='cuda:1')
current relative error is:  tensor(0.9360, device='cuda:1')
current epoch is:  3350
current loss is:  tensor(-3.7379, device='cuda:1')
current relative error is:  tensor(0.9257, device='cuda:1')
current epoch is:  3400
current loss is:  tensor(-7.3364, device='cuda:1')
current relative error is:  tensor(0.9390, device='cuda:1')
current epoch is:  3450
current loss is:  tensor(-5.421

current epoch is:  6150
current loss is:  tensor(-7.9941, device='cuda:1')
current relative error is:  tensor(0.9727, device='cuda:1')
current epoch is:  6200
current loss is:  tensor(-6.6244, device='cuda:1')
current relative error is:  tensor(0.9270, device='cuda:1')
current epoch is:  6250
current loss is:  tensor(-7.3079, device='cuda:1')
current relative error is:  tensor(0.9197, device='cuda:1')
current epoch is:  6300
current loss is:  tensor(-5.2181, device='cuda:1')
current relative error is:  tensor(0.9679, device='cuda:1')
current epoch is:  6350
current loss is:  tensor(-4.7278, device='cuda:1')
current relative error is:  tensor(0.9501, device='cuda:1')
current epoch is:  6400
current loss is:  tensor(-7.0179, device='cuda:1')
current relative error is:  tensor(0.9447, device='cuda:1')
current epoch is:  6450
current loss is:  tensor(-2.9305, device='cuda:1')
current relative error is:  tensor(0.9427, device='cuda:1')
current epoch is:  6500
current loss is:  tensor(-6.750

current epoch is:  9200
current loss is:  tensor(-6.7825, device='cuda:1')
current relative error is:  tensor(0.9122, device='cuda:1')
current epoch is:  9250
current loss is:  tensor(-10.3053, device='cuda:1')
current relative error is:  tensor(0.8958, device='cuda:1')
current epoch is:  9300
current loss is:  tensor(-8.6820, device='cuda:1')
current relative error is:  tensor(0.9066, device='cuda:1')
current epoch is:  9350
current loss is:  tensor(-10.1573, device='cuda:1')
current relative error is:  tensor(0.9064, device='cuda:1')
current epoch is:  9400
current loss is:  tensor(-3.7679, device='cuda:1')
current relative error is:  tensor(0.8944, device='cuda:1')
current epoch is:  9450
current loss is:  tensor(-9.8751, device='cuda:1')
current relative error is:  tensor(0.9073, device='cuda:1')
current epoch is:  9500
current loss is:  tensor(-6.7397, device='cuda:1')
current relative error is:  tensor(0.9411, device='cuda:1')
current epoch is:  9550
current loss is:  tensor(-7.4

current epoch is:  12250
current loss is:  tensor(-12.1601, device='cuda:1')
current relative error is:  tensor(0.8279, device='cuda:1')
current epoch is:  12300
current loss is:  tensor(-17.6819, device='cuda:1')
current relative error is:  tensor(0.8248, device='cuda:1')
current epoch is:  12350
current loss is:  tensor(-16.6287, device='cuda:1')
current relative error is:  tensor(0.8501, device='cuda:1')
current epoch is:  12400
current loss is:  tensor(-13.7872, device='cuda:1')
current relative error is:  tensor(0.8309, device='cuda:1')
current epoch is:  12450
current loss is:  tensor(-14.3941, device='cuda:1')
current relative error is:  tensor(0.8213, device='cuda:1')
current epoch is:  12500
current loss is:  tensor(-17.2241, device='cuda:1')
current relative error is:  tensor(0.8110, device='cuda:1')
current epoch is:  12550
current loss is:  tensor(-13.6521, device='cuda:1')
current relative error is:  tensor(0.8209, device='cuda:1')
current epoch is:  12600
current loss is:

...

current epoch is:  87250
current loss is:  tensor(-76.9432, device='cuda:1')
current relative error is:  tensor(0.0561, device='cuda:1')
current epoch is:  87300
current loss is:  tensor(-74.8132, device='cuda:1')
current relative error is:  tensor(0.1083, device='cuda:1')
current epoch is:  87350
current loss is:  tensor(-73.0680, device='cuda:1')
current relative error is:  tensor(0.1146, device='cuda:1')
current epoch is:  87400
current loss is:  tensor(-81.3601, device='cuda:1')
current relative error is:  tensor(0.1219, device='cuda:1')
current epoch is:  87450
current loss is:  tensor(-84.6600, device='cuda:1')
current relative error is:  tensor(0.0661, device='cuda:1')
current epoch is:  87500
current loss is:  tensor(-81.8904, device='cuda:1')
current relative error is:  tensor(0.0514, device='cuda:1')
current epoch is:  87550
current loss is:  tensor(-79.4827, device='cuda:1')
current relative error is:  tensor(0.0558, device='cuda:1')
current epoch is:  87600
current loss is:

current epoch is:  90250
current loss is:  tensor(-77.7281, device='cuda:1')
current relative error is:  tensor(0.0616, device='cuda:1')
current epoch is:  90300
current loss is:  tensor(-79.9746, device='cuda:1')
current relative error is:  tensor(0.0523, device='cuda:1')
current epoch is:  90350
current loss is:  tensor(-77.3644, device='cuda:1')
current relative error is:  tensor(0.0992, device='cuda:1')
current epoch is:  90400
current loss is:  tensor(-73.7713, device='cuda:1')
current relative error is:  tensor(0.1360, device='cuda:1')
current epoch is:  90450
current loss is:  tensor(-80.3103, device='cuda:1')
current relative error is:  tensor(0.0739, device='cuda:1')
current epoch is:  90500
current loss is:  tensor(-75.4483, device='cuda:1')
current relative error is:  tensor(0.0585, device='cuda:1')
current epoch is:  90550
current loss is:  tensor(-81.9868, device='cuda:1')
current relative error is:  tensor(0.1294, device='cuda:1')
current epoch is:  90600
current loss is:

current epoch is:  93250
current loss is:  tensor(-73.3733, device='cuda:1')
current relative error is:  tensor(0.0575, device='cuda:1')
current epoch is:  93300
current loss is:  tensor(-78.2103, device='cuda:1')
current relative error is:  tensor(0.0575, device='cuda:1')
current epoch is:  93350
current loss is:  tensor(-81.9349, device='cuda:1')
current relative error is:  tensor(0.0541, device='cuda:1')
current epoch is:  93400
current loss is:  tensor(-78.2831, device='cuda:1')
current relative error is:  tensor(0.0878, device='cuda:1')
current epoch is:  93450
current loss is:  tensor(-77.6793, device='cuda:1')
current relative error is:  tensor(0.0580, device='cuda:1')
current epoch is:  93500
current loss is:  tensor(-82.1661, device='cuda:1')
current relative error is:  tensor(0.0624, device='cuda:1')
current epoch is:  93550
current loss is:  tensor(-73.7455, device='cuda:1')
current relative error is:  tensor(0.0775, device='cuda:1')
current epoch is:  93600
current loss is:

current epoch is:  96250
current loss is:  tensor(-77.4667, device='cuda:1')
current relative error is:  tensor(0.0860, device='cuda:1')
current epoch is:  96300
current loss is:  tensor(-77.1416, device='cuda:1')
current relative error is:  tensor(0.0855, device='cuda:1')
current epoch is:  96350
current loss is:  tensor(-84.0635, device='cuda:1')
current relative error is:  tensor(0.0531, device='cuda:1')
current epoch is:  96400
current loss is:  tensor(-81.4931, device='cuda:1')
current relative error is:  tensor(0.0707, device='cuda:1')
current epoch is:  96450
current loss is:  tensor(-80.9229, device='cuda:1')
current relative error is:  tensor(0.1004, device='cuda:1')
current epoch is:  96500
current loss is:  tensor(-81.2938, device='cuda:1')
current relative error is:  tensor(0.0653, device='cuda:1')
current epoch is:  96550
current loss is:  tensor(-74.7036, device='cuda:1')
current relative error is:  tensor(0.1019, device='cuda:1')
current epoch is:  96600
current loss is:

current epoch is:  99250
current loss is:  tensor(-82.5582, device='cuda:1')
current relative error is:  tensor(0.0762, device='cuda:1')
current epoch is:  99300
current loss is:  tensor(-80.6274, device='cuda:1')
current relative error is:  tensor(0.0807, device='cuda:1')
current epoch is:  99350
current loss is:  tensor(-79.0183, device='cuda:1')
current relative error is:  tensor(0.0460, device='cuda:1')
current epoch is:  99400
current loss is:  tensor(-81.4099, device='cuda:1')
current relative error is:  tensor(0.0632, device='cuda:1')
current epoch is:  99450
current loss is:  tensor(-69.5647, device='cuda:1')
current relative error is:  tensor(0.1166, device='cuda:1')
current epoch is:  99500
current loss is:  tensor(-81.3686, device='cuda:1')
current relative error is:  tensor(0.0540, device='cuda:1')
current epoch is:  99550
current loss is:  tensor(-83.3548, device='cuda:1')
current relative error is:  tensor(0.0617, device='cuda:1')
current epoch is:  99600
current loss is: